In [5]:
from obspy import read
st = read()  # load example seismogram
st.filter(type='highpass', freq=3.0)
st = st.select(component='Z')
#st.plot()

In [7]:
import webbrowser
url = "https://www.sciencebase.gov/catalog/item/655bebe7d34ee4b6e05cc19f"

# Open the URL in the default web browser
webbrowser.open(url)

True

In [4]:
import os
# KMl File Download
usgs_fdb_url = "https://www.sciencebase.gov/catalog/item/655bebe7d34ee4b6e05cc19f"
kml_file = "NSHM23_FSD_v3.kml"
geojson_file = "NSHM23_FSD_v3.geojson"

cmd = f"wget '{usgs_fdb_url}?name={kml_file}' -O ./{kml_file}"
print(cmd)
!{cmd}

cmd2 = f"wget '{usgs_fdb_url}?name={geojson_file}' -O ./{geojson_file}"
print(cmd2)
!{cmd2}

wget 'https://www.sciencebase.gov/catalog/item/655bebe7d34ee4b6e05cc19f?name=NSHM23_FSD_v3.kml' -O ./NSHM23_FSD_v3.kml
--2024-12-08 16:34:57--  https://www.sciencebase.gov/catalog/item/655bebe7d34ee4b6e05cc19f?name=NSHM23_FSD_v3.kml
Resolving www.sciencebase.gov (www.sciencebase.gov)... 137.227.248.21
Connecting to www.sciencebase.gov (www.sciencebase.gov)|137.227.248.21|:443... connected.
HTTP request sent, awaiting response... 200 200
Length: unspecified [text/html]
Saving to: ‘./NSHM23_FSD_v3.kml’

./NSHM23_FSD_v3.kml     [  <=>               ]  48.45K   242KB/s    in 0.2s    

2024-12-08 16:34:58 (242 KB/s) - ‘./NSHM23_FSD_v3.kml’ saved [49615]

wget 'https://www.sciencebase.gov/catalog/item/655bebe7d34ee4b6e05cc19f?name=NSHM23_FSD_v3.geojson' -O ./NSHM23_FSD_v3.geojson
--2024-12-08 16:34:58--  https://www.sciencebase.gov/catalog/item/655bebe7d34ee4b6e05cc19f?name=NSHM23_FSD_v3.geojson
Resolving www.sciencebase.gov (www.sciencebase.gov)... 137.227.248.21
Connecting to www.scienceba

In [ ]:
import os
import json
import time
import itertools
import datetime
import cartopy
# Third-party Imports
import numpy
import numpy as np
from scipy import ndimage as nd
import matplotlib.pyplot as plt
import matplotlib.pyplot as pyplot
import matplotlib.pyplot as plot
import matplotlib.patches as mpatches
import csep
import pandas as pd
import libcomcat
from libcomcat.dataframes import get_history_data_frame, split_history_frame, PRODUCTS
from libcomcat.search import get_event_by_id

In [ ]:
from csep.utils import datasets, time_utils, comcat, plots
from csep.core import regions, catalog_evaluations

# set start and end date
start_time = time_utils.strptime_to_utc_datetime('2010-04-08 00:00:00.0')
end_time = time_utils.strptime_to_utc_datetime('2010-04-09 23:59:59.0')
# retrieve events in ComCat catalogue between start and end date
catalog = csep.query_comcat(start_time, end_time)
min_mw = 3.95 # minimum magnitude
max_mw = 8.95 # max magnitude after which is just one bin
dmw = 0.1 # bin width

# Create space and magnitude regions. The forecast is already filtered in space and magnitude
magnitudes = regions.magnitude_bins(min_mw, max_mw, dmw)
region = regions.california_relm_region()

# Bind region information to the forecast 
space_magnitude_region = regions.create_space_magnitude_region(region, magnitudes)


# filter magnitude below 3.95
catalog.filter('magnitude >= 3.95')
# filter events outside spatial region
catalog.filter_spatial(space_magnitude_region)
# print summary information 
print(catalog)

#######################################
# CAGALOGUE SHOULD CONTAIN 10 EVENTS ##
#######################################

In [ ]:
#load forecast
forecast = csep.load_catalog_forecast('u3_fore_2010_04_08.bin',
                                      start_time = start_time,
                                      end_time = end_time,
                                      type='ucerf3',
                                      region = space_magnitude_region,
                                      filter_spatial = True,
                                      apply_filters = True,
                                      filters = 'magnitude >= 3.95')

In [ ]:
evt_counts = forecast.get_event_counts()

In [ ]:
##############################
# Thia sum should be 420734 ##
##############################
np.sum(evt_counts)

In [ ]:
# compute N-test
number_test_result = catalog_evaluations.number_test(forecast, catalog)

In [ ]:
##################################################################
## results should be delta_1 = 0.08, delta_2 = 0.93, omega = 10 ##
################################################################## 
ax = number_test_result.plot()

In [ ]:
#calculate expected rates per space-magnitude bin
expected_rates = forecast.get_expected_rates(verbose=True)

In [ ]:
args_forecast = {'title': 'Landers aftershock forecast',
                 'grid_labels': True,
                 'borders': True,
                 'feature_lw': 0.5,
                 'basemap': 'ESRI_imagery',
                 'cmap': 'rainbow',
                 'alpha_exp': 0.9,
                 'projection': cartopy.crs.Mercator(),
                 'clim':[-9, 0]}
args_catalog = {'basemap': 'ESRI_terrain',
                'markercolor': 'black',
                'markersize': 4}
ax_1 = expected_rates.plot(plot_args=args_forecast)
ax_2 = catalog.plot(ax=ax_1, plot_args=args_catalog)
#########################################################
# Image only has observations in the right side corner ##
#########################################################